# Diffusion simulation classes

> Ricardo Peres, 06.05.2022

In [ ]:
import numpy as np
import warnings
import scipy.integrate
import scipy.interpolate
import pickle

#optional
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.patches import Circle
from matplotlib.patches import Rectangle
from tqdm import tqdm

#non-trivial packages
from hexalattice.hexalattice import create_hex_grid
import numba

: 

In [ ]:
class TPC:
    '''
    General TPC class. Contains useful constants, common variables 
    and functions relatesd to the overall system.
    '''
    
    def __init__(self, radius, length, liquid_gap, gas_gap, drift_field):
        self.radius = radius #radius of TPC
        self.length = length #lenght of liquid bellow gate
        self.liquid_gap = liquid_gap #distance from gate to liquid-gas interface
        self.gas_gap = gas_gap #distance from liquid-gas interface to sensors
        self.drift_field = drift_field #V/cm
        
        self.z_gate = 0 # should come from config
        self.liquid_level = self.z_gate + self.liquid_gap
        self.z_anode = self.liquid_level + self.gas_gap
        
        self.drift_velocity = self.model_velocity(self.drift_field) #mm/us
        self.diffusion_long = self.model_diffusion_longitudinal(self.drift_field) #mm2/us
        self.diffusion_trans = self.model_diffusion_transversal(self.drift_field) #mm2/us
        
        self.mesh_pitch = 1.56 # should come from config
        self.gate_mesh = MeshGrid(self.radius, self.mesh_pitch)
    
    @classmethod
    def model_velocity(self, drift_field):
        '''
        Drift velocity model from NEST, implemented by Yanina 
        Biondi (https://github.com/YaniBion).
        Given a drift_field values [V/cm] returns the expected electron drift 
        velocity in LXe. In mm/us
        '''
        #par = [-3.1046, 27.037, -2.1668, 193.27, -4.8024, 646.04, 9.2471]
        par = [-1.5000, 28.510, -.21948, 183.49, -1.4320, 1652.9, 2.884]
        dv = (par[0] * np.exp(-drift_field/par[1]) + 
              par[2] * np.exp(-drift_field/par[3]) + 
              par[4] * np.exp(-drift_field/par[5]) +
              par[6]) #mm/us
        return dv
    
    @classmethod
    def model_diffusion_longitudinal(self, drift_field):
        '''
        Longitudinal diffusion model: TO DO.
        For 100 V/cm -> ~26cm2/us from 1T
        '''
        if drift_field != 100:
            warnings.warn('Only the long. diffusino value for 100 V/cm is implemented, going with it for now.')
        D_l = 26 #cm2/s
        return D_l * 1e-4 #mm2/us
    
    @classmethod
    def model_diffusion_transversal(self, drift_field):
        '''
        Transversal diffusion model. Linear fit interpolation from the EXO200 paper data (Fig. 7) [
        Curve fit: m=0.010635; b=52.888942
        Linregress: m=0.013021; b=52.752949
        '''
        m=0.010635
        b=52.888942
        ans = m * drift_field + b #cm2/s
        return ans * 1e-4 #mm2/us
    
    @staticmethod
    def get_r(x,y):
        '''Get r from x and y.'''
        return np.sqrt(np.power(x,2) + np.power(y,2))
    
    @staticmethod
    def get_theta(x,y):
        '''Get theta from x and y.'''
        theta = np.arctan(y/x)
        if (x < 0) and (y < 0):
            theta = -theta
        return theta
    
    @staticmethod
    def get_xyz(r,theta,phi):
        '''Convert to cartesian following Physics standard spherical 
        coordinates.'''
        x = r*np.cos(phi)*sin(theta)
        y = r*np.sin(phi)*sin(theta)
        z = r*np.cos(theta)
        return x,y,z
    
    @staticmethod
    def simple_warning(message, category, filename, lineno, file=None, line=None):
        return 'WARNING: %s\n' %message
    
    

: 

In [ ]:
class MeshGrid:
    '''
    Grid-related stuff. Construct, change, focus, etc.
    These helpstrings are becoming worse and worse, I know.
    '''
    
    def __init__(self, r_max, hex_side):
        self.r_max = r_max
        self.hex_side = hex_side
        self.hex_r = self.hex_side * np.sqrt(3)
        
        self.hex_centers = self.construct_mesh()
        self.n_hexes = len(self.hex_centers)
        
    def construct_mesh(self):
        '''
        Construct the hexagonal gate mesh middles.
          * r_max: radius of the electrode
          * a: size of the hexagons
        '''
        
        n_hex_x = 2 * np.ceil(75/self.hex_r)
        n_hex_y = 2 * np.ceil(75/(np.sqrt(2)*self.hex_side))
        
        hex_centers, _ = create_hex_grid(nx=n_hex_x, ny=n_hex_y, 
                                         min_diam = self.hex_side,
                                         crop_circ = self.r_max,
                                         do_plot = False)
        
        return hex_centers
    
    def distance_to_reference_grid(self,pos_xy):
        '''
        Given an array of reference positions (hex grid in this partivular case) - format 
        [[xs,ys]] -, return the distance of a point (x0,y0) to these reference points.
        pos_xy is a 1d 2 value array.
        '''
        hex_distance = np.sqrt(np.power(self.hex_centers[:,0]-pos_xy[0],2) + 
                               np.power(self.hex_centers[:,1]-pos_xy[1],2)
                              )
        return hex_distance

    def get_closest_hex(self, pos_xy, value = True):
        '''
        Given a reference grid and a point on the plane, return the grid index where 
        the point is closeste to a grid point.
        '''
        distances = self.distance_to_reference_grid(pos_xy)
        idx_min_dist = np.argmin(distances)
        if value == True:
            return self.hex_centers[idx_min_dist]
        else:
            return idx_min_dist
    
    def focus_on_grid(self,pos_array):
        '''
        Focuses the electrons to the center of the hexagons. `pos_array` must
        be of the shape (N, 2), where pos_array[:,0] are x values and 
        pos_array[:,1] are y values.
        Returns a (N,2) array of the positions after the focus effect.
        '''
        pos_focus = np.apply_along_axis(self.get_closest_hex, 1, pos_array)
        return pos_focus
    
    def count_focused(self, pos):
        '''
        Counts the number of events on each hex center.
        '''
        N_counts = np.zeros(self.n_hexes)
        for _hex_i, _hex_xy in tqdm(enumerate(self.hex_centers),
                                    'Counting hits in hex centers',
                                    total = self.n_hexes):
            N_counts[_hex_i] = len(np.where((pos[:,0] == _hex_xy[0]) &
                                            (pos[:,1] == _hex_xy[1])
                                           )[0])
        assert np.sum(N_counts) == len(pos), 'Lots some electrons counting. Woops!'
        return N_counts

: 

In [ ]:
class XeLamp:
    '''
    Properties and functions of the Xe lamp and its brightly 
    shining effects on producing electrons on the photocathode.
    '''
    
    def __init__(self,delta_t_lamp):
        self.numerical_aperture = 0.22
        self.distance2photok = 2 # mm
        self.length_xy = self.numerical_aperture*self.distance2photok
        
        #time step to consider when reconstructing the lamp pulse
        #use 2 ns (0.002 us) to match ADC freq or 0.25 for testing 
        #(from YB's original code)
        self.delta_t_lamp = delta_t_lamp 
        self.times_lamp = np.arange(0,6,self.delta_t_lamp)
        
    @classmethod
    def pulse_lamp(cls,t):
        '''
        Parametrization of electrons emitted by a pulse of the lamp.
        '''
        calc = 6e4*np.exp(-(t-2.8)**2/2/(2.90/2.355)**2 )
        return calc
    
    def emitted_electrons_in_interval(self,t0,tf, error = False):
        '''
        Integrate the lamp pulse to number of electrons from t0 to tf.
        '''
        integral = scipy.integrate.quad(XeLamp.pulse_lamp,t0,tf,epsrel = 1e-6)
        
        if error==True:
            
            return int(integral[0]),integral[1]
        else:
            return int(integral[0])
        
    
    def init_positions(self, n_electrons, shape = 'circle'):
        '''
        Initial spread of electrons on x,y and z.
        Standard version uses a Gaussian spread over x and y 
        with sigma given by the lamp aperture.
        '''
        if shape != 'circle':
            warnings.warn('Not implemented yet. Taking circle.')
            
        sigma_xy = self.numerical_aperture * self.distance2photok
        mu, sigma = 0, np.sqrt(sigma_xy) # mean and standard deviation
        X0 = np.random.normal(mu, sigma, n_electrons)
        Y0 = np.random.normal(mu,sigma, n_electrons)
        #Z0 = np.random.normal(mu,1e-3, n_electrons)
        Z0 = np.zeros(n_electrons)
        return X0,Y0,Z0

: 

In [ ]:
class ElectronDrift:
    '''
    Processes and variables related to teh drift of electrons through the LXe TPC.
    Needs the input of a TPC object and an int n_electrons
    '''
    
    def __init__(self, tpc, xelamp,drift_delta_t):
        self.tpc = tpc
        self.drift_velocity = tpc.drift_velocity 
        self.drift_delta_t = drift_delta_t # us
        
        self.sigma_trans = np.sqrt(2*tpc.diffusion_trans*self.drift_delta_t)
        self.sigma_long = np.sqrt(2*tpc.diffusion_long*self.drift_delta_t)
        
        self.lamp = xelamp
        
        #harcoded values to go to a config file
        self.e_lifetime = 2000 #us
        self.se_gain = 28.57 #pe/e- from Xurich ii
        self.extract_efficiency = 0.99 # Electron extraction efficiency
        
        warnings.formatwarning = TPC.simple_warning
    
    def check_boundaries(self):
        '''
        Check if all the electrons are within the boundaries.
        If outside r>r_max -> turn to nan
        If at z=0 (gate) -> put into X/Y/Z_gas
        '''
        mask_at_gate = Z >= 0
        
        t_gas[mask_at_gate] = self.t
        
        R = get_r(X,Y)
        mask_outside_bound = R > self.TPC.radius
        X[outside_bound_mask] = np.nan
        Y[outside_bound_mask] = np.nan
        Z[outside_bound_mask] = np.nan
    
    def drift_lamp_pulse_slice(self, t0_pulse_slice, tf_pulse_slice):
        '''
        Drift electrons from a given integration period of the lamp pulse.
        '''
        
        n_electrons = lamp.emitted_electrons_in_interval(t0_pulse_slice, tf_pulse_slice)
        x0,y0,z0 = self.lamp.init_positions(n_electrons)
        x,y,z = x0.copy(),y0.copy(),z0.copy()
        
        self.t_running = 0
        warnings.warn('Starting drifting process with dt=%.2f us. Light-speed!'%self.drift_delta_t)
        half_warning = False
        while np.any(z<self.tpc.length): 
            #there is a catch here that I'm not recording the x,y 
            #when they cross z=tpc.length but only when ALL cross z=tpc.length
            # TO BE FIXED
            x,y,z = self.drift_step(x,y,z)
            
            if np.any(z>self.tpc.length/2) and half_warning==False:
                warnings.warn('An electron reached half-way there in %d us.' %self.t_running)
                half_warning = True
                
        warnings.warn('All electrons have drifted to the gate after %d us.' %self.t_running)
        
        return x,y,z
    
    def drift_full_pulse(self, lamp_end_time = 6):
        delta_t_lamp = lamp.delta_t_lamp
        times_lamp = np.arange(0,8, lamp.delta_t_lamp)
        
        positions_lamp_slices = dict() # bulky but practicle
        
        for t_lamp_index in tqdm(range(len(times_lamp) - 1)):
            x,y,z = self.drift_lamp_pulse_slice(times_lamp[t_lamp_index],
                                                times_lamp[t_lamp_index + 1])
            positions_lamp_slices['slice_%d'%t_lamp_index] = dict(start = times_lamp[t_lamp_index],
                                                                  end = times_lamp[t_lamp_index + 1],
                                                                  delta_t_lamp = delta_t_lamp,
                                                                  x = x,
                                                                  y = y,
                                                                  z = z)
        return positions_lamp_slices
    
    def drift_step(self, x,y,z):
        '''
        Make an increment on the electron array position following diffusion.
        Currently the arrival times are not recorded.
        '''
        _n_electrons = len(x)
        delta_x = np.random.normal(0, self.sigma_trans, _n_electrons)
        delta_y = np.random.normal(0, self.sigma_trans, _n_electrons)
        delta_z = (np.random.normal(0, self.sigma_long, _n_electrons) +
                   self.drift_velocity * self.drift_delta_t)
        
        x = x + delta_x
        y = y + delta_y
        z = z + delta_z
        
        self.t_running += self.drift_delta_t
        return x,y,z

    def apply_corrections(self, x,y,z):
        '''
        Apply all the reductions needed.
        '''
        x,y,z = self.apply_elifetime(x,y,z)
        x,y,z = self.extract_electrons(x,y,z)
        
        return x,y,z
    
    def extract_electrons(self,x,y,z):
        '''
        Apply extraction efficiency.
        '''
        n_electrons = len(x)
        n_unlucky = round(n_electrons * (1-self.extract_efficiency))
        unlucky_electrons_index = np.random.choice(n_electrons,size = n_unlucky, replace=False)  
        
        x = np.delete(x, unlucky_electrons_index)
        y = np.delete(y, unlucky_electrons_index)
        z = np.delete(z, unlucky_electrons_index)
        # #turn to nan the unlucky electrons
        # x[unlucky_electrons_index] = np.nan
        # y[unlucky_electrons_index] = np.nan
        # z[unlucky_electrons_index] = np.nan
        
        return x,y,z
    
    def apply_elifetime(self,x,y,z):
        '''
        Reduce the ammount of electrons due to the non-infinite e-lifetime.
        To finish: x,y,z_old are the final arrays of electrons.
        ''' 
        n_electrons = len(x)
        drift_time = self.tpc.length / self.drift_velocity # should be electron dependant in the future
       
        n_unlucky = round(n_electrons *(1-np.exp(-drift_time/self.e_lifetime)))
        unlucky_electrons_index = np.random.choice(n_electrons,size = n_unlucky, replace=False)  
        
        x = np.delete(x, unlucky_electrons_index)
        y = np.delete(y, unlucky_electrons_index)
        z = np.delete(z, unlucky_electrons_index)
        
        # #turn to nan the unlucky electrons
        # x[unlucky_electrons_index] = np.nan
        # y[unlucky_electrons_index] = np.nan
        # z[unlucky_electrons_index] = np.nan
        
        return x,y,z
    
    def convert_electron_to_photons(self,n_electrons):
        '''
        This will be a very fancy method to infer the number of either photons or 
        pe or whatever. For now it uses the SE gain value of either Xurich II.
        Please make a PR.
        '''
        
        n_pe = n_electrons * self.se_gain
        
        return n_pe
    

: 

In [ ]:
class LCEPattern:
    '''
    Ad-hoc LCE maps. From pos and number of electrons to photons detected.
    '''
    
    def __init__(self, TPC):
        
        # TO BE CLEANED UP
        self.TPC = TPC
        self.z_gate = TPC.z_gate
        self.z_anode = TPC.z_anode
        self.gate_mesh = TPC.gate_mesh
        self.r_max = TPC.radius
        
        #self.load_patterns()
    
    def define_pattern_props(self, x_bin_step, y_bin_step, n_traces, smooth_pattern, force_traces = False):
        '''
        Call this function to define the pattern calculation properties.
        '''
        
        self.x_bin_step = 1 #mm #to put in config
        self.y_bin_step = 1 #mm #to put in config
        self.pattern_bin_area = self.x_bin_step * self.y_bin_step
        if force_traces != True:
            assert n_traces < 1e7, 'Asked for too many traces. Proceed on your own accord with force_traces = True'
        self.n_traces = int(n_traces)
        self.smooth_pattern = smooth_pattern
        return None
    
    #@classmethod
    def get_xy_on_plane(self,x0,y0,z0,directions,z_prime):
        '''
        Return the (x,y) of the intersection of a vector with direction 
        (theta,phi), physics convention, starting at point (x0,y0,z0) and
        finishing at the plane z=z_prime.
        '''
        x_prime = x0 + (z_prime-z0) * np.cos(directions[:,1]) * np.tan(directions[:,0])
        y_prime = y0 + (z_prime-z0) * np.sin(directions[:,1]) * np.tan(directions[:,0])
        final_pos = np.stack((x_prime, y_prime), axis = 1)
        return final_pos
    
    #@classmethod
    def get_xy_on_circ_array(self, final_pos):
        final_r = TPC.get_r(final_pos[:,0],final_pos[:,1])
        mask_r = final_r < self.r_max
        final_pos_array = final_pos[mask_r]
        return final_pos_array
    
    #@classmethod
    def get_hits_on_circ_array(self, x0,y0,z0):
        '''
        Get the positions of the toys that hit the circular area of the array.
        '''
        
        thetas = np.random.uniform(0,np.pi/2,self.n_traces)
        phis = np.random.uniform(0,2*np.pi,self.n_traces)
        directions = np.stack((thetas,phis), axis = 1)
        
        final_pos = self.get_xy_on_plane(x0,y0,z0,directions, self.z_anode)
        final_pos_array = self.get_xy_on_circ_array(final_pos)
        return final_pos_array
    
    #@classmethod
    def print_stats_of_hits(self, hits, n_traces):
        print('Initial number of photons: %s'%n_traces)
        print('Number of photons hit: '+
              '%s (%.2f%% of produced, %.2f of '+
              'full emission)'%(len(hits),
                                len(hits)/n_traces*100,
                                len(hits)/n_traces*100/2))
        return None
    
    #@classmethod
    #def load_patterns(self,redo_patterns):
    #    '''
    #    Load the interpolated functions of the pattern for a set of points.
    #    Checks if they already exist and if `redo=True` to compute new ones
    #    '''
    #    pass
    #   
    #    if redo_patterns == True:
    #        #for each hex center:
    #        pattern = self.make_pattern_density(x_bin_step, y_bins_step)
    #        with open('patterns/hex_%d.pck', 'wb') as file:
    #            pickle.dump(pattern, file)
    
    #make this @classmethod??
    def get_pattern_density_hist2d(self, pos):
        
        x_min = y_min = -np.ceil(self.r_max*1.1)
        x_max = y_max = np.ceil(self.r_max*1.1)
        
        x_bin_sides = np.arange(x_min,x_max+self.x_bin_step*0.1,self.x_bin_step)
        y_bin_sides = np.arange(y_min,y_max+self.x_bin_step*0.1,self.y_bin_step)
        
        x_bin_middles = x_bin_sides[:-1] + self.x_bin_step/2
        y_bin_middles = y_bin_sides[:-1] + self.y_bin_step/2
        
        hist2d = np.histogram2d(pos[:,0],pos[:,1],
                        bins = (x_bin_sides,y_bin_sides),
                                density = False)
        #density could be True but we can also do it by hand given the bin area 
        #and total of photons. It's even better because it can be normalized properly 
        #from the start taking into account the photonos projected downwards and the
        #ones that miss the array.
        assert np.sum(hist2d[0]) == len(pos), ('Lost some photons on the histogram??\n'+
                                               'N_toys: %d\n' %len(pos)+
                                               'N_hist2d: %d' %len(hist2d[2]))
        
        
        #total_in_hist = np.sum(hist2d[0])
        #double the traces innormalization because of under side of sphere
        hist_density = hist2d[0]/self.pattern_bin_area/(self.n_traces*2) #fraction/mm^2;
        return x_bin_middles,y_bin_middles,hist_density
    
    # make @classmethod ??
    def make_pattern_density(self,pos):
        '''
        Takes the toy-MC results and makes the 2D density histogram, 
        normalized to the total number of traces produced and bin area.
        Returns either the interpolative function.
        Params:
          * pos: (N,2) array of the positions of hits in the circular array
          * x_bin_step: size of the hist2d bin length in x
          * y_bin_step: size of the hist2d bin length in y
        '''
        if self.smooth_pattern:
            s = 0.0000001
        else:
            s = 0
        x_bin_middles,y_bin_middles,hist_density = self.get_pattern_density_hist2d(pos)
        interp2s = scipy.interpolate.RectBivariateSpline(x_bin_middles, 
                                                         y_bin_middles,
                                                         hist_density,
                                                         s = s)
        return interp2s
    
    @staticmethod
    def plot_pattern(pattern, hex_id):
        fig,ax = plt.subplots(1,1,figsize = (9,9), dpi = 100)
        ax.set_title('Pattern interpolation\n(spline, k=3)')
        _x = np.arange(-80,80,1)
        _y = np.arange(-80,80,1)
        _xx,_yy = np.meshgrid(_x,_y, indexing='ij')
        _rr = TPC.get_r(_xx,_yy)
        _xx = _xx[_rr < Xenoscope.radius]
        _yy = _yy[_rr < Xenoscope.radius]
        _zz = pattern.ev(_xx,_yy)
        interpolated = ax.scatter(_xx, _yy, c=np.log10(_zz), marker = 's',
                                  s = 3, vmin = -6.2)

        ax.add_patch(Circle((0,0),75, color = 'r',fill = False, linewidth = 1, ls ='--'))
        ax.set_aspect('equal')
        fig.colorbar(interpolated, ax = ax)

        plt.savefig('figures/patterns/hex_v0_%d' %hex_id)

: 

In [ ]:
class TopArray:
    def __init__(self, tpc, mesh,model,path_to_patterns):
        self.model = model
        self.tpc = tpc
        self.path_to_patterns = path_to_patterns
        self.mesh = mesh
        
        #self.load_top_array()
        self.make_results_grid()
        
        
    def make_results_grid(self, x_step=1, y_step=1):
        self.grid_x_step = x_step
        self.grid_y_step = y_step
        self.grid_x_max = self.tpc.radius
        self.grid_y_max = self.tpc.radius
        self.grid_x_min = -self.tpc.radius
        self.grid_y_min = -self.tpc.radius
        self.grid_x = np.arange(self.grid_x_min, 
                                self.grid_x_max,
                                self.grid_x_step)
        self.grid_y = np.arange(self.grid_y_min,
                                self.grid_y_max,
                                self.grid_y_step)
        self.grid_xx,self.grid_yy = np.meshgrid(self.grid_x,
                                                self.grid_y,
                                                indexing='ij')
        self.grid_zz = np.zeros((self.grid_yy.shape))
        return None
    
    def load_top_array(self):
        with open('TopArrayModel/%s.pck'%self.model, 'rb') as file:
            self.sensors = pickle.load(file)
        self.n_sensors = len(self.sensors)
        
        return None
    
    def plot_toparray(self,ax, pe_in_sensors = False):
        ax.set_aspect('equal')
        for _quad_i,_quad in enumerate(self.sensors):
            xy = (_quad[0],_quad[2])
            width = _quad[1]-_quad[0]
            height = _quad[3]-_quad[2]
            if pe_in_sensors == False:
                ax.add_patch(Rectangle(xy,width,height, 
                                       fill = False, 
                                       color = 'k'))
            else:
                #TO DO
                #check if results exist and default to False
                
                cm = plt.get_cmap('gnuplot')
                log_pe = np.log10(self.pe_in_sensors)
                results_max = np.max(log_pe)
                results_min = np.min(log_pe[np.isfinite(log_pe)])
                log_pe = np.nan_to_num(log_pe, copy = True,neginf=results_min)
                results_max = np.max(log_pe)
                results_min = np.min(log_pe)
                pe = log_pe[_quad_i]

                ax.add_patch(Rectangle(xy,width,height, 
                                       fill = True, 
                                       edgecolor = 'k',
                                       facecolor = cm((pe-results_min)/
                                                      (results_max-results_min)),
                                      )
                            )
        ax.set_ylim(-90,90)
        ax.set_xlim(-90,90)
        ax.set_aspect('equal')
        ax.set_xlabel('x [mm]')
        ax.set_ylabel('y [mm]')
        ax.add_patch(Circle((0,0),75, color = 'r',fill = False))
        return ax
    
    def integrate_in_quad(self,quad):
        ans = self.results_interp.integral(quad[0],quad[1],quad[2],quad[3])
        return ans
    
    def load_pattern(self, hex_id):
        with open(self.path_to_patterns + 'hex_v0_%d.pck'%hex_id, 'rb') as file:
            pattern = pickle.load(file)
        return pattern
    
    def load_all_patterns(self):
        patterns = dict()
        for _hex_i in tqdm(range(self.mesh.n_hexes),
                           'Loading LCE patterns.',
                           total = self.mesh.n_hexes):
            patterns[_hex_i] = self.load_pattern(_hex_i)
        
        self.patterns = patterns
        
        return None
    
    def fill_grid_from_events(self, counts_pe_on_hex):
        '''
        Sum over all the patterns of LCE x number of pe per hex to
        get the final pe pattern in the array.
        '''
        
        for hex_i in tqdm(range(self.mesh.n_hexes),
                          'Summing all normalized patterns. 1+1+2+3+5+8+...',
                          total = self.mesh.n_hexes):
            #pattern = load_pattern(self, hex_i)
            pattern = self.patterns[hex_i]
            if counts_pe_on_hex[hex_i] > 0:
                _zz = pattern.ev(self.grid_xx,self.grid_yy) * counts_pe_on_hex[hex_i]
                self.grid_zz += _zz
            else:
                continue
        
        self.results_interp = scipy.interpolate.RectBivariateSpline(self.grid_x, 
                                                                    self.grid_y,
                                                                    self.grid_zz,
                                                                    s = 0)
        return None
    
    def n_pe_in_sensors(self):
        pe_in_sensors = np.apply_along_axis(self.integrate_in_quad,
                                          1, 
                                          self.sensors)
        self.pe_in_sensors = pe_in_sensors
        return pe_in_sensors
        

: 

: 